# IMPORT

In [1]:
import gc
import glob
import mne
import numpy as np
import pandas as pd


# UTILE

In [2]:
class Config :
    def init(self):
        return
cfg = Config()

In [3]:
def mne_plot(x) : del x
def num_digits(__number):
    return len(str(__number))
def accu_fn(__ok_loss):
    return (
        lambda __output, __target: ((__output - __target).abs() < __ok_loss)
        .float()
        .mean()
        .item()
    )
def calculate_metrics(confusion_matrix):
    # Unpacking the confusion matrix
    [[TP, FP], [FN, TN]] = confusion_matrix

    # Calculating metrics
    accuracy = (TP + TN) / (TP + FP + FN + TN)
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

    return {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1_score,
        "CM"  : confusion_matrix
    }
def evaluate_the_classification(
    actual_labels, predicted_labels, num_classes, tempeture=900
):
    cm = confusion_matrix(
            actual_labels.detach().to("cpu"),
            predicted_labels.detach().argmax(dim=1).to("cpu"),
            labels=range(num_classes),
    )
    if num_classes == 2:
        return calculate_metrics(cm)

    # Calculate true positives, true negatives, false positives, and false negatives
    cm = torch.tensor(cm)
    TP = torch.diag((cm))
    FP = cm.sum(axis=0) - TP
    FN = cm.sum(axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)

    # Calculate accuracy
    accuracy = (TP.sum() + TN.sum()) / (TP.sum() + TN.sum() + FP.sum() + FN.sum())

    # Calculate precision, recall, and F1 score for each class
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1_score = 2 * (precision * recall) / (precision + recall)

    # Calculate average precision, recall, and F1 score
    avg_precision = precision.mean()
    avg_recall = recall.mean()
    avg_f1_score = f1_score.mean()

    return {
        "CM": cm,
        "CM-norm": (cm / tempeture).softmax(dim=0),
        "Accuracy": accuracy.item(),
        "Precision": precision.tolist(),
        "Recall": recall.tolist(),
        "F1 Score": f1_score.tolist(),
        "Average Precision": avg_precision.item(),
        "Average Recall": avg_recall.item(),
        "Average F1 Score": avg_f1_score.item(),
    }
def init_THE_LOG(model_name , LOG) :
        # THE LOGGIN
    if LOG is None :
        LOG = {}
    def add_to_LOG(__str):
        if str(__str) not in LOG :
            LOG[str(__str)] = []

    for kind in ('LOSS-Train' , "ACCU-Train" , "LOSS-Test" , "ACCU-Test") :
            add_to_LOG(f"{kind}")
    return LOG

# PREP-DATA

In [4]:
cfg.SEQ_LEN = 2000
cfg.__dict__

{'SEQ_LEN': 2000}

In [19]:
!shutdown 

Shutdown scheduled for Sun 2024-10-27 02:26:27 GMT, use 'shutdown -c' to cancel.


In [5]:
for person in range(55):
    paths = glob.glob(f"EEG-DATA/**s{person:02}_**.set/**s{person:02}_**.set")
    print(paths)
    for path in paths:
        data = {
            "sequence" : [], 
            "reaction-time" : [], 
            "correction-time" : [] 
        }
        try:
            eeg_data = mne.io.read_raw_eeglab(path, preload=False)
            events, event_ids = mne.events_from_annotations(eeg_data)

            DL_on = event_ids["251"]
            DR_on = event_ids["252"]
            R_on = event_ids["253"]
            R_off = event_ids["254"]
        except KeyError as e:
            print(f"Error: Missing event_id {e} for person {person:02}")
            continue
        except Exception as e:
            print(f"Error reading data for {person:02}: {e}")
            continue
        t_DON , t_ROFF , t_RON = 0 , 0 , 0
        values, counts = np.unique(events[:, 2], return_counts=True)
        if not (
                counts[DL_on - 1] + counts[DR_on - 1] == counts[R_on - 1]
                and counts[DL_on - 1] + counts[DR_on - 1] == counts[R_off - 1]
            ):
            continue
        del values, counts 
        for index, (e_time, _, e_type) in enumerate(events):
            if e_type == DL_on or e_type == DR_on:
                start_time = max(0, e_time - cfg.SEQ_LEN)
                raw_data_segment = eeg_data.get_data(picks="FP1")[:, start_time:e_time][0]
                data["sequence"].append(raw_data_segment.tolist())
                del raw_data_segment
                gc.collect()
                t_DON = e_time
            elif e_type == R_on:
                t_RON = e_time
                data['reaction-time'].append(t_RON - t_DON)
            elif e_type == R_off:
                t_ROFF = e_time
                data['correction-time'].append(t_ROFF - t_RON)
        
        
        from pathlib import Path

        __path = Path(path)
        base_name = __path.stem  
        pd.DataFrame(data).to_csv(f"FP1/{base_name}.csv" , index=False)

[]
['EEG-DATA/s01_060926_2n.set/s01_060926_2n.set', 'EEG-DATA/s01_051017m.set/s01_051017m.set', 'EEG-DATA/s01_060227n.set/s01_060227n.set', 'EEG-DATA/s01_060926_1n.set/s01_060926_1n.set', 'EEG-DATA/s01_061102n.set/s01_061102n.set']
Reading /home/mohammedaouamri/DEV/leurn/EEG/EEG-DATA/s01_060926_2n.set/s01_060926_2n.fdt
Used Annotations descriptions: [np.str_('251'), np.str_('252'), np.str_('253'), np.str_('254')]
Reading /home/mohammedaouamri/DEV/leurn/EEG/EEG-DATA/s01_051017m.set/s01_051017m.fdt
Used Annotations descriptions: [np.str_('251'), np.str_('252'), np.str_('253'), np.str_('254')]
Reading /home/mohammedaouamri/DEV/leurn/EEG/EEG-DATA/s01_060227n.set/s01_060227n.fdt
Used Annotations descriptions: [np.str_('251'), np.str_('252'), np.str_('253'), np.str_('254')]
Reading /home/mohammedaouamri/DEV/leurn/EEG/EEG-DATA/s01_060926_1n.set/s01_060926_1n.fdt
Used Annotations descriptions: [np.str_('251'), np.str_('252'), np.str_('253'), np.str_('254')]
Reading /home/mohammedaouamri/DEV/le